In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    break
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
root_dir = "/kaggle/input/corn-leaf-infection-dataset/Corn Disease detection"
os.listdir(root_dir)

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
# hyper parameters
batch_size = 32
img_height, img_width=128,128
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    root_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    root_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

In [ ]:
validation_generator[0][1][1]

In [ ]:
def show(img, title=None, fig_size=(5, 5)):
    fig = plt.figure(figsize=fig_size)
    if title is not None:
        plt.title(title)
    plt.imshow(img)
    plt.axis("off")
    plt.show()

In [ ]:
import cv2
plt.style.use('seaborn-whitegrid')
for i in range(3):
    labels = ["Normal", "Infected"]
    img = validation_generator[0][0][i]
    lbl = labels[np.argmax(validation_generator[0][1][i])]
    
    img = cv2.resize(img, (img_height, img_width))
    img = img.reshape(img_height, img_width, 3)
    show(img, title=lbl)

In [ ]:
from keras.applications.inception_resnet_v2 import preprocess_input, InceptionResNetV2
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from keras.models import Model

img_input = Input(shape=(img_height, img_width, 3), name = 'grayscale_input_layer')
# x = Conv2D(3, (3,3),  padding= 'same', name = 'grayscale_RGB_layer')(img_input)
inception = InceptionResNetV2(input_shape = (img_height, img_width, 3), include_top = False, weights = 'imagenet')
inception.layers.pop()  # Remove classification layer

for layer in inception.layers:
    layer.trainable = False
    if type(layer) == 'BatchNormalization':
        layer.momentum = 1.0

inception = inception(img_input)
output = inception
output = GlobalAveragePooling2D()(output)

output = Dropout(0.5)(output)
output = Dense(2, activation="softmax")(output) 
model = Model(inputs=[img_input], outputs=[output], name='embedding_model')

In [ ]:
lr=1e-03
epochs=15

opt = Adam(lr=lr, decay=lr / epochs)

# compile it !
model.compile(loss="categorical_crossentropy", optimizer=opt,	metrics=["accuracy"])

# train it
history=model.fit_generator(
	train_generator,
	steps_per_epoch=len(train_generator),
	validation_data=validation_generator,
	validation_steps=len(validation_generator),
	epochs=epochs,
  verbose=1,
	workers=32)

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()